In [11]:
import json
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, Image, HTML
import random
import pandas as pd

In [12]:
dataset = "OVQA"
balanced = False
answer_type = "OPEN"

In [13]:
scores_path = Path(f"/home/kckahl/E132-Projekte/Projects/2024_Erkan_Kahl_VLMRobustness/Experiments_Cluster_Sweep/{dataset}/ia3/llava-{dataset}_train_all-finetune_ia3_lr3e-2_seed123/eval/{dataset}_val_all/mistral_metrics.json")
with open(scores_path, 'r') as f:
    scores = json.load(f)
scores_df = pd.DataFrame(scores[1:]).set_index("qid")
scores_df_equal = scores_df.loc[scores_df["gt"] == scores_df["pred"]]
scores_df_non_equal = scores_df.loc[scores_df["gt"] != scores_df["pred"]]
print(len(scores_df_non_equal.loc[scores_df_non_equal["mistral_score"] == 1]))
print(len(scores_df_non_equal.loc[scores_df_non_equal["mistral_score"] == 2]))
print(len(scores_df_non_equal.loc[scores_df_non_equal["mistral_score"] == 3]))
print(len(scores_df_non_equal.loc[scores_df_non_equal["mistral_score"] == 4]))
print(len(scores_df_non_equal.loc[scores_df_non_equal["mistral_score"] == 5]))

262
109
55
41
25


In [14]:
if balanced:
    rater_study_path = f"/nvme/VLMRobustness/{dataset}/rater_study_open.json"
    rater_study_non_balanced = pd.read_json(rater_study_path)
    rater_study_non_balanced = rater_study_non_balanced.set_index("qid")
    scores_rater_study_non_balanced = scores_df.loc[scores_df.index.isin(rater_study_non_balanced.index)]
    scores_rater_study_non_balanced

In [15]:
from collections import Counter

base_path = Path(f"/nvme/VLMRobustness/{dataset}")

json_file_path = base_path / "validate.json"
if balanced:
    output_file_path = base_path / f"rater_study_{answer_type.lower()}_balanced.json"
else:
    output_file_path = base_path / f"rater_study_{answer_type.lower()}.json"

with open(json_file_path, 'r') as f:
    data = json.load(f)
#data = [element for element in data if element.get("q_lang")=="en"]
data = [element for element in data if element.get("answer_type")==answer_type]
if balanced:
    data = [element for element in data if element.get("qid") in list(scores_df_non_equal.index) and element.get("qid") not in list(rater_study_non_balanced.index)]
else:
    data = [element for element in data if element.get("qid") in list(scores_df_non_equal.index)]
print(len(data))

# Initialize index and max index
current_index = 0
max_index = len(data)
selected_question_ids = []
selected_questions = []
selected_questions_scores = []

def display_current_question():
    global current_index
    add_button.disabled = False
    remove_button.disabled = True
    element = data[current_index]
    if element["qid"] in selected_question_ids:
        add_button.disabled = True
        remove_button.disabled = False
    id_text.value = ""
    question_widget.value = f"<b>Question:</b> {element['question']}<br><b>Answer:</b> {element['answer']}<br><b>QID:</b> {element['qid']}"

def on_next_button_clicked(b):
    global current_index, max_index
    current_index = (current_index + 1) % max_index
    display_current_question()

def on_previous_button_clicked(b):
    global current_index, max_index
    if current_index == 0:
        return
    current_index = (current_index - 1) % max_index
    display_current_question()

def on_random_button_clicked(b):
    global current_index, max_index
    current_index = random.randint(0, max_index)
    display_current_question()

def on_add_button_clicked(b):
    global current_index, max_index, selected_questions, selected_question_ids
    element = data[current_index]
    selected_question_ids.append(element["qid"])
    selected_questions.append(element)
    selected_questions_scores.append(scores_df.loc[element["qid"]]["mistral_score"])
    add_button.disabled = True
    remove_button.diabled = False
    display_selected_questions()

def on_remove_button_clicked(b):
    global current_index, max_index, selected_questions, selected_question_ids
    element = data[current_index]
    selected_question_ids.remove(element["qid"])
    selected_questions = [e for e in selected_questions if e["qid"]!=element["qid"]]
    add_button.disabled = False
    remove_button.disabled = True
    display_selected_questions()

def on_id_button_clicked(b):
    global current_index, max_index
    question = [(index, element) for index, element in enumerate(data) if str(element["qid"]) == id_text.value]
    if len(question) == 1:
        current_index = question[0][0]
        display_current_question()
    else:
        print(f"couldn't find question {id_text.value}")

def on_random_balanced_clicked(b):
    global current_index
    score_counter = Counter(selected_questions_scores)
    less_than_four = [score for score in range(1,6) if score not in score_counter.keys() or score_counter[score] < 4 or score_counter[score] == len(scores_df_non_equal.loc[scores_df_non_equal["mistral_score"] == score])]
    if len(less_than_four) == 0:
        print("No scores left with less than 4 selected questions")
    if len(less_than_four) == 1 and less_than_four[0] == 5:
        df_5_non_equal = scores_df_non_equal.loc[scores_df_non_equal["mistral_score"] == 5]
        if len(df_5_non_equal) == score_counter[5]:
            print("No more non-equal 5 scores left and no other scores left with less than 4 selected questions")
    random_score = random.choice(less_than_four)
    random_qid = scores_df_non_equal.loc[(scores_df_non_equal["mistral_score"] == random_score) & ~scores_df_non_equal.index.isin(rater_study_non_balanced.index)].sample().index.values[0]
    question = [(index, element) for index, element in enumerate(data) if element["qid"] == random_qid]
    current_index = question[0][0]
    display_current_question()

def on_save_clicked(b):
    with open(output_file_path, 'w') as f:
        data = json.dump(selected_questions, f, indent=4)
    print("saved")


question_widget = widgets.HTML()
previous_button = widgets.Button(description="Previous")
next_button = widgets.Button(description="Next")
random_button = widgets.Button(description="Random Sample")
add_button = widgets.Button(description="Add Sample to Eval")
remove_button = widgets.Button(description="Remove Sample from Eval", enabled=False)
id_text = widgets.Text(
    value='',
    placeholder='',
    disabled=False
)
id_button = widgets.Button(description="Jump to ID")
save_button = widgets.Button(description="Save questions to file")

next_button.on_click(on_next_button_clicked)
previous_button.on_click(on_previous_button_clicked)
if balanced:
    random_button.on_click(on_random_balanced_clicked)
else:
    random_button.on_click(on_random_button_clicked)
add_button.on_click(on_add_button_clicked)
remove_button.on_click(on_remove_button_clicked)
id_button.on_click(on_id_button_clicked)
save_button.on_click(on_save_clicked)

# Display the first image and information
display_current_question()

# Layout the widgets
buttons_previous_next_random = widgets.HBox([previous_button, next_button, random_button])
select_id = widgets.HBox([id_text, id_button])
add_remove_button = widgets.HBox([add_button, remove_button])
ui = widgets.VBox([question_widget, buttons_previous_next_random, select_id, add_remove_button, save_button])
display(ui)

selected_questions_widget = widgets.HTML()

def display_selected_questions():
    selected_questions_widget.value = f"================<br><b>SELECTED QUESTIONS (total {len(selected_questions)})</b><br>" +  "<br>================<br>".join(f"<b>Question:</b> {element['question']}<br><b>Answer:</b> {element['answer']}<br><b>QID:</b> {element['qid']}" for element in selected_questions)

display(selected_questions_widget)

492


HTML(value='')